In [ ]:
from google.colab import files
uploaded = files.upload()

- Name : 게임의 이름입니다.
- Platform : 게임이 지원되는 플랫폼의 이름입니다.
- Year : 게임이 출시된 연도입니다.
- Genre : 게임의 장르입니다.
- Publisher : 게임을 제작한 회사입니다.
- NA_Sales : 북미지역에서의 출고량입니다.
- EU_Sales : 유럽지역에서의 출고량입니다.
- JP_Sales : 일본지역에서의 출고량입니다.
- Other_Sales : 기타지역에서의 출고량입니다.
### 필수 내용
- 지역에 따라서 선호하는 게임 장르가 다를까 라는 질문에 대답을 하셔야합니다.
- 연도별 게임의 트렌드가 있을까 라는 질문에 대답을 하셔야합니다.
- 출고량이 높은 게임에 대한 분석 및 시각화 프로세스가 포함되어야 합니다

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('/content/vgames2_1.csv', index_col= 0)
#df = df.sort_values(by=  , ascending=False) /특정컬럼기준으로 내림차순 정렬, (ascending)오름차순
#df.shape /(16598, 9)
df.head()
# Year 결측지 4000으로 수정, 형태에 맞지 않는 값 형태 4자리 맞추기

#year = df['Year'].isnull().sum() / 271 개의 결측치
df['Year'] = df['Year'].fillna(4000)
#year = df['Year'].isnull().sum() / 0 결측치 제거 완료

#y_s = df['Year'].sum() /33643998.0  형태 변환전 숫치

df['Year'] = pd.to_numeric(df['Year'], downcast='integer')

def year_form(x) :
  if 0 <= x <= 21 : 
    x = x + 2000
  elif 22 <= x <= 99 :
    x = x + 1900
  else :
    x = x
  return x

df['Year'] = df['Year'].apply(year_form)

'''
결측지 4000으로 바뀌었는지 확인
year_check = df.sort_values(by='Year' , ascending=False)
year_check
#y_s = df['Year'].sum() /33842598 형태 수정후 합
'''

In [ ]:
'''
데이터 폼 변경후 오류 체크 이상무
df = df[df.Year != 4000]
df = df.sort_values(by='Year' , ascending=False)
df
'''

In [ ]:
# 장르 결측치 = m_g, miss genre / 50개의 결측치
df['Genre'] = df['Genre'].fillna('m_g')

# 제작사 결측치 = m_p, miss publisher /203개의 결측치
df['Publisher'] = df['Publisher'].fillna('m_p')
df.info()


게임 출하량은 기준은 K로 둔다, 여기서 K와 M가 나오지만 m을 기준을 해버리면 100단위 수준의 판매량이 너무나 많이 나오게 되는데 국가 단위의 누적 출하량이 말도 않되게 작다, 그래서 K의 기준으로 데이터를 분석한다.

In [ ]:
df.info()

In [ ]:
#K 제거
df['NA_Sales'] = df['NA_Sales'].str.replace('K','')
df['EU_Sales'] = df['EU_Sales'].str.replace('K','')
df['JP_Sales'] = df['JP_Sales'].str.replace('K','')
df['Other_Sales'] = df['Other_Sales'].str.replace('K','')

#M 을 기준을 판별
df['NA_Sales_M'] = df['NA_Sales'].str.contains('M')
df['EU_Sales_M'] = df['EU_Sales'].str.contains('M')
df['JP_Sales_M'] = df['JP_Sales'].str.contains('M')
df['Other_Sales_M'] = df['Other_Sales'].str.contains('M')

df['NA_Sales_M'] = df['NA_Sales_M'].replace({True: 10**3, False: 10**6})
df['EU_Sales_M'] = df['EU_Sales_M'].replace({True: 10**3, False: 10**6})
df['JP_Sales_M'] = df['JP_Sales_M'].replace({True: 10**3, False: 10**6})
df['Other_Sales_M'] = df['Other_Sales_M'].replace({True: 10**3, False: 10**6})

#M 제거
df['NA_Sales'] = df['NA_Sales'].str.replace('M','')
df['EU_Sales'] = df['EU_Sales'].str.replace('M','')
df['JP_Sales'] = df['JP_Sales'].str.replace('M','')
df['Other_Sales'] = df['Other_Sales'].str.replace('M','')

In [ ]:
df.info()

In [ ]:
df[['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']] = df[['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']].apply(pd.to_numeric)

In [ ]:
# 단위 곱하기
df['NA_Sales'] = df['NA_Sales'].multiply(df['NA_Sales_M'])
df['EU_Sales'] = df['EU_Sales'].multiply(df['EU_Sales_M'])
df['JP_Sales'] = df['JP_Sales'].multiply(df['JP_Sales_M'])
df['Other_Sales'] = df['Other_Sales'].multiply(df['Other_Sales_M'])


In [ ]:
df['NA_Sales'] = df['NA_Sales'].astype(int)
df['EU_Sales'] = df['EU_Sales'].astype(int)
df['JP_Sales'] = df['JP_Sales'].astype(int)
df['Other_Sales'] = df['Other_Sales'].astype(int)

In [ ]:
#df 정리
df = df.loc[:,['Name', 'Platform', 'Year', 'Genre', 'Publisher', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']]

In [ ]:
df.head(10)

In [ ]:
  df.to_csv("vgames2_EDA.csv")

In [ ]:
#sum 은 4개지역의 출하량 합
df['area_sum'] =  df['NA_Sales'] + df['EU_Sales'] + df['JP_Sales'] + df['Other_Sales']
df.head()

In [ ]:
df = df.sort_values(by= 'JP_Sales'  , ascending=False)
df.head(50)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
#지역에 따라서 선호하는 게임 장르
area_NA = df #copy
area_NA = area_NA['NA_Sales'].groupby(area_NA['Genre']).sum()
area_EU = df #copy
area_EU = area_EU['EU_Sales'].groupby(area_EU['Genre']).sum()
area_JP = df #copy
area_JP = area_JP['NA_Sales'].groupby(area_JP['Genre']).sum()
area_Other = df #copy
area_Other = area_Other['Other_Sales'].groupby(area_Other['Genre']).sum()
area_sum = df #copy
area_sum = area_sum['area_sum'].groupby(area_sum['Genre']).sum()

#내림 차순 정렬
area_NA = area_NA.sort_values(ascending=False)
area_EU = area_EU.sort_values(ascending=False)
area_JP = area_JP.sort_values(ascending=False)
area_Other = area_Other.sort_values(ascending=False)
area_sum = area_sum.sort_values(ascending=False)

In [ ]:
genre_list = ['Action', 'Racing', 'm_g', 'Sports', 'Misc', 'platform', 'Shooter', 'Simulation', 'Role-Playing', 'Fighting', 'Strategy', 'Puzzle', 'Adventure']
fig = plt.figure(figsize=(8, 8))
fig.set_facecolor('white')
ax = fig.add_subplot()
ax.pie(x=area_NA, labels = genre_list, autopct = lambda p :'{:.2f}%'.format(p))
plt.show()

In [ ]:
fig = plt.figure(figsize=(8, 8))
fig.set_facecolor('white')
ax = fig.add_subplot()
ax.pie(x=area_EU, labels = genre_list, autopct = lambda p :'{:.2f}%'.format(p))
plt.show()

In [ ]:
fig = plt.figure(figsize=(8, 8))
fig.set_facecolor('white')
ax = fig.add_subplot()
ax.pie(x=area_JP, labels = genre_list, autopct = lambda p :'{:.2f}%'.format(p))
plt.show()

In [ ]:
fig = plt.figure(figsize=(8, 8))
fig.set_facecolor('white')
ax = fig.add_subplot()
ax.pie(x=area_Other, labels = genre_list, autopct = lambda p :'{:.2f}%'.format(p))
plt.show()

In [ ]:
fig = plt.figure(figsize=(8, 8))
fig.set_facecolor('white')
ax = fig.add_subplot()
ax.pie(x=area_sum, labels = genre_list, autopct = lambda p :'{:.2f}%'.format(p))
plt.show()

In [ ]:
area_sum = df[df['Year'].isin(['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'])] #copy
area_sum = area_sum['area_sum'].groupby(area_sum['Genre']).sum()


fig = plt.figure(figsize=(8, 8))
fig.set_facecolor('white')
ax = fig.add_subplot()
ax.pie(x=area_sum, labels = genre_list, autopct = lambda p :'{:.2f}%'.format(p))
plt.show()

In [ ]:
area_sum = df[df['Year'].isin(['2015', '2016', '2017', '2018', '2019', '2020'])] #copy
area_sum = area_sum['area_sum'].groupby(area_sum['Genre']).sum()


fig = plt.figure(figsize=(8, 8))
fig.set_facecolor('white')
ax = fig.add_subplot()
ax.pie(x=area_sum, labels = genre_list, autopct = lambda p :'{:.2f}%'.format(p))
plt.show()

In [ ]:
#)연도별 게임의 출시트렌드

#area_NA = area_NA['NA_Sales'].groupby(area_NA['Genre']).sum()

In [ ]:
area_sum = df[df['Year'].isin(['2016', '2017', '2018', '2019', '2020'])] #copy
#area_sum = area_sum['area_sum'].groupby(area_sum['Genre']).sum()
area_sum = area_sum['Genre']
genre_list = ['Action', 'Racing', 'm_g', 'Sports', 'Misc', 'platform', 'Shooter', 'Simulation', 'Role-Playing', 'Fighting', 'Strategy', 'Puzzle', 'Adventure']
'''
fig = plt.figure(figsize=(8, 8))
fig.set_facecolor('white')
ax = fig.add_subplot()
ax.pie(x=area_sum, labels = genre_list, autopct = lambda p :'{:.2f}%'.format(p))
plt.show()
'''

In [ ]:
genre_list = ['Action', 'Racing', 'm_g', 'Sports', 'Misc', 'platform', 'Shooter', 'Simulation', 'Role-Playing', 'Fighting', 'Strategy', 'Puzzle', 'Adventure']

g_count = df['Genre'].value_counts()
g_count

In [ ]:
y1980 = df[df['Year'].isin(['1980'])]
y1981 = df[df['Year'].isin(['1981'])]
y1982 = df[df['Year'].isin(['1982'])]
y1983 = df[df['Year'].isin(['1983'])]
y1984 = df[df['Year'].isin(['1984'])]
y1985 = df[df['Year'].isin(['1985'])]
y1986 = df[df['Year'].isin(['1986'])]
y1987 = df[df['Year'].isin(['1987'])]
y1988 = df[df['Year'].isin(['1988'])]
y1989 = df[df['Year'].isin(['1989'])]
y1990 = df[df['Year'].isin(['1990'])]
y1991 = df[df['Year'].isin(['1991'])]
y1992 = df[df['Year'].isin(['1992'])]
y1993 = df[df['Year'].isin(['1993'])]
y1994 = df[df['Year'].isin(['1994'])]
y1995 = df[df['Year'].isin(['1995'])]
y1996 = df[df['Year'].isin(['1996'])]
y1997 = df[df['Year'].isin(['1997'])]
y1998 = df[df['Year'].isin(['1998'])]
y1999 = df[df['Year'].isin(['1999'])]
y2000 = df[df['Year'].isin(['2000'])]
y2001 = df[df['Year'].isin(['2001'])]
y2002 = df[df['Year'].isin(['2002'])]
y2003 = df[df['Year'].isin(['2003'])]
y2004 = df[df['Year'].isin(['2004'])]
y2005 = df[df['Year'].isin(['2005'])]
y2006 = df[df['Year'].isin(['2006'])]
y2007 = df[df['Year'].isin(['2007'])]
y2008 = df[df['Year'].isin(['2008'])]
y2009 = df[df['Year'].isin(['2009'])]
y2010 = df[df['Year'].isin(['2010'])]
y2011 = df[df['Year'].isin(['2011'])]
y2012 = df[df['Year'].isin(['2012'])]
y2013 = df[df['Year'].isin(['2013'])]
y2014 = df[df['Year'].isin(['2014'])]
y2015 = df[df['Year'].isin(['2015'])]
y2016 = df[df['Year'].isin(['2016'])]
y2017 = df[df['Year'].isin(['2017'])]
y2018 = df[df['Year'].isin(['2018'])]
y2019 = df[df['Year'].isin(['2019'])]
y2020 = df[df['Year'].isin(['2020'])]
y4000 = df[df['Year'].isin(['4000'])]

In [ ]:
y_g1980 = y1980['Genre'].value_counts()
y_g1981 = y1980['Genre'].value_counts()
y_g1982 = y1980['Genre'].value_counts()
y_g1983 = y1980['Genre'].value_counts()
y_g1984 = y1980['Genre'].value_counts()
y_g1985 = y1980['Genre'].value_counts()
y_g1986 = y1980['Genre'].value_counts()
y_g1987 = y1980['Genre'].value_counts()
y_g1988 = y1980['Genre'].value_counts()
y_g1989 = y1980['Genre'].value_counts()
y_g1990 = y1980['Genre'].value_counts()
y_g1991 = y1980['Genre'].value_counts()
y_g1992 = y1980['Genre'].value_counts()
y_g1993 = y1980['Genre'].value_counts()
y_g1994 = y1980['Genre'].value_counts()
y_g1995 = y1980['Genre'].value_counts()
y_g1996 = y1980['Genre'].value_counts()
y_g1997 = y1980['Genre'].value_counts()
y_g1998 = y1980['Genre'].value_counts()
y_g1999 = y1980['Genre'].value_counts()
y_g2000 = y2000['Genre'].value_counts()
y_g2001 = y2001['Genre'].value_counts()
y_g2002 = y2002['Genre'].value_counts()
y_g2003 = y2003['Genre'].value_counts()
y_g2004 = y2004['Genre'].value_counts()
y_g2005 = y2005['Genre'].value_counts()
y_g2006 = y2006['Genre'].value_counts()
y_g2007 = y2007['Genre'].value_counts()
y_g2008 = y2008['Genre'].value_counts()
y_g2009 = y2009['Genre'].value_counts()
y_g2010 = y2010['Genre'].value_counts()
y_g2011 = y2011['Genre'].value_counts()
y_g2012 = y2012['Genre'].value_counts()
y_g2013 = y2013['Genre'].value_counts()
y_g2014 = y2014['Genre'].value_counts()
y_g2015 = y2015['Genre'].value_counts()
y_g2016 = y2016['Genre'].value_counts()
y_g2017 = y2017['Genre'].value_counts()
y_g2018 = y2018['Genre'].value_counts()
y_g2019 = y2019['Genre'].value_counts()
y_g2020 = y2020['Genre'].value_counts()
y_g4000 = y4000['Genre'].value_counts()


In [ ]:
y_g = pd.concat([y_g1980,y_g1981,y_g1982,y_g1983,y_g1984,y_g1985,y_g1986, y_g1987, y_g1988, y_g1989, y_g1990, y_g1991, y_g1992, y_g1993, y_g1994, y_g1995, y_g1996, y_g1997, y_g1998, y_g1999, y_g2000, y_g2001, y_g2002, y_g2003, y_g2004, y_g2005, y_g2006, y_g2007, y_g2008, y_g2009, y_g2010, y_g2011, y_g2012, y_g2013, y_g2014, y_g2015, y_g2016, y_g2017, y_g2018, y_g2019, y_g2020, y_g4000,g_count], axis=1)

y_g.columns = ['1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999','2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '4000', 'sum']
y_g = y_g.rename(index={'':'gemre'}  )
y_g = y_g.fillna(value= 0)
y_g = y_g.astype(int)
y_g.head(13)


In [ ]:
y_g_s = pd.concat([y_g2000, y_g2001, y_g2002, y_g2003, y_g2004, y_g2005, y_g2006, y_g2007, y_g2008, y_g2009, y_g2010, y_g2011, y_g2012, y_g2013, y_g2014, y_g2015, y_g2016, y_g2017, y_g2020, y_g4000,g_count], axis=1)

y_g_s.columns = ['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2020', '4000', 'sum']
y_g_s = y_g_s.rename(index={'':'gemre'}  )
y_g_s = y_g_s.fillna(value= 0)
y_g_s = y_g_s.astype(int)
y_g_s.head(13)
